In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import random
import time
from duckduckgo_search import DDGS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH = '/content/drive/My Drive/rackitt-drug-discovery/crawler/'

## Google Search

In [3]:
sites = ['ncbi', 'Sciencedirect', 'Researchgate', 'Journal plos', 'springer', 'elsevier']
ingredients = ['Vitamin D3']#['whole milk', 'Maltodextrin', 'Cocoa', 'sugar', 'Flavours', 'vitamin E', 'vitamin B3', 'Vitamin D3', 'Vitamin A','polyphenols',
              #  'vitamin K1', 'vitamin B1', 'vitamin B5', 'vitamin B7', 'vitamin B2', 'Vitamin B12', 'vitamin B6', 'Vitamin B9','ZINC SULFATE',
              #  'MANGANESE SULPHATE', 'CUPRIC SULFATE', 'SODIUM MOLYBDATE', 'POTASSIUM IODIDE', 'SODIUM SELENITE', 'CHROMIC CHLORIDE', 'TRICALCIUM PHOSPHATE']

interaction_list = ['Interactions', 'Reactions', 'Oxidation', 'Reduction', 'Precipitation', 'Absorption','Degradation',
         'Agglomeration', 'Off-notes', 'Color']

# groupings = ['vitamins', 'minerals', 'whole milk powder'] #['polyphenols/phenolic acid']
# interaction_list = ['Interactions', 'Reactions', 'Oxidations', 'Reductions', 'Preciptation', 'Absorption','Degradation',
#          'Aglomeration', 'Off-notes', 'Colour fading', 'Colour changing']
main_compound = 'iron'


In [5]:
def get_search_query(site_name, interaction, compound):
    # Replace 'YOUR_SEARCH_QUERY' with your actual search query
    if site_name==None:
        site_name = ''
    search_query = 'research papers and article on "{query}" between "'+ main_compound+ ' " and "{grouping}" "{site}"'
    search_query = search_query.format(grouping=compound, site = site_name, query = interaction)
    url = f"https://www.google.com/search?q={search_query}"
    return url

def extract_links_title(links):

    link_list = []
    title_list = []
    for link in links:
        link_href = link.get('href')
        if "url?q=" in link_href and not "webcache" in link_href:
            title = link.find_all('h3')
            if len(title) > 0:
                link_list.append(link.get('href').split("?q=")[1].split("&sa=U")[0])
                title_list.append(title[0].getText())
    return link_list, title_list

def get_search(url):
    # Perform the request
    params = {
            "hl": "en",         # language
            "num": "20"         # number of results given 10 links per page
            }

    # proxies = get_proxy()
    sesh = requests.Session()
    response = sesh.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all("a")
    link_list, title_list = extract_links_title(links)
    return link_list, title_list

def get_dha_compund_link(site_name, query, compound):

    url = get_search_query(site_name, query, compound)
    link_list, title_list = get_search(url)
    site_name_list = [site_name]*len(link_list)
    return link_list, title_list, site_name_list

In [7]:
for compound in ingredients:
    link_df = pd.DataFrame()
    for interaction in tqdm.tqdm(interaction_list):
        for site_name in sites:
            link_list, title_list, site_name_list = get_dha_compund_link(site_name, interaction, compound)
            df = pd.DataFrame(zip(link_list, title_list, site_name_list), columns = ['link','title', 'source-site'])
            df['compound'] = compound
            df['interaction'] = interaction
            df['correct_link_source'] = df['link'].apply(lambda x: site_name.lower().split()[0] in x)
            if site_name=='elsevier':
              df['correct_link_source']=1
            df = df[df['correct_link_source']==1]
            link_df = pd.concat([link_df, df])
            # time.sleep(random.randint(3,10))
    link_df = link_df.drop_duplicates(subset='link')
    link_df.to_excel(PATH+'ferrous/google/extracted_links_ferrous_'+compound+'.xlsx', index=False)

100%|██████████| 10/10 [00:39<00:00,  3.97s/it]


In [8]:
link_df

,link,title,source-site,compound,interaction,correct_link_source
0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Iron bioavailability of a casein-based iron fo...,ncbi,whole milk,Interactions,1
1,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,Safety of iron milk proteinate as a novel food...,ncbi,whole milk,Interactions,1
2,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Impact of Ascorbic Acid on the In Vitro Iron B...,ncbi,whole milk,Interactions,1
3,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,Physicochemical Properties and Oxidative Stabi...,ncbi,whole milk,Interactions,1
4,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,Effects of Iron Supplementation on Metabolism ...,ncbi,whole milk,Interactions,1
...,...,...,...,...,...,...
13,https://www.researchgate.net/publication/33659...,Effect of forage type in the ovine diet on the...,Researchgate,whole milk,Off-notes,1
14,https://www.researchgate.net/publication/23015...,"Descriptive Profile Analysis of Cooked, Stored...",Researchgate,whole milk,Off-notes,1
16,https://www.researchgate.net/publication/46403...,(PDF) Investigating age-related changes in tas...,Researchgate,whole milk,Off-notes,1
17,https://www.researchgate.net/publication/34613...,Preliminary investigation on the effect of pro...,Researchgate,whole milk,Off-notes,1


## Duckduckgo search

In [ ]:
sites = ['ncbi', 'Sciencedirect', 'Researchgate', 'Journal plos', 'springer', 'elsevier']
ingredients = ['polyphenols']#['milk', 'Maltodextrin', 'Cocoa', 'sugar', 'Flavours', 'vitamin E', 'vitamin B3', 'Vitamin D3', 'Vitamin A','polyphenols',
              #  'vitamin K1', 'vitamin B1', 'vitamin B5', 'vitamin B7', 'vitamin B2', 'Vitamin B12', 'vitamin B6', 'Vitamin B9','ZINC SULFATE',
              #  'MANGANESE SULPHATE', 'CUPRIC SULFATE', 'SODIUM MOLYBDATE', 'POTASSIUM IODIDE', 'SODIUM SELENITE', 'CHROMIC CHLORIDE', 'TRICALCIUM PHOSPHATE']

interaction_list = ['Oxidation', 'Reduction', 'Precipitation', 'Absorption','Degradation',
         'Agglomeration', 'Off-notes', 'Color']
main_compounds = ['iron']


In [ ]:
for compound in tqdm.tqdm(ingredients):
    link_df_duck = pd.DataFrame()
    for main_compound in main_compounds:

        for interaction in tqdm.tqdm(interaction_list):
            for site_name in sites:

                search_query = 'research papers and article on "{interaction}" between "{main_compound}" and "{compound}" {site}'
                search_query = search_query.format(interaction = interaction.lower(),
                                                   main_compound = main_compound.lower(),
                                                   compound = compound.lower(),
                                                   site = site_name.lower())

                results = DDGS().text(search_query, max_results=30)
                time.sleep(random.randint(3,10))
                df = pd.DataFrame(results)
                df['main_compound'] = main_compound
                df['interaction'] = interaction
                df['source'] = site_name
                link_df_duck = pd.concat([link_df_duck, df])

    link_df_duck['interaction_compound'] =  compound
    link_df_duck = link_df_duck.drop_duplicates(subset='href')
    link_df_duck = link_df_duck.drop_duplicates(subset='title')
    link_df_duck.drop('body', axis=1).to_excel(PATH+'ferrous/duckduckgo/duck_duck_go_extracted_links_ferrous_'+compound+'.xlsx', index=False)

#### Prompt

In [ ]:
'''
You are a very helpful assisant.
It is very important to know the positive and negative effects due to interaction between DHA/docosahexaenoic acid and other chemical compounds like vitamin B12, Vitamin E, vitamin D etc.
Interaction between DHA and other chemical compounds impacts the oxidation, solubility, stability and Micronutrients of the DHA.
I will give you the website link of research papers and article. You please go through the website and give me information on the impacts oxidation, solubility, stability and microneutrients.
Please also give 4-5 line summary of the website.

For example -  if the website is https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4727338/, it mentions the below impacts:

Oxidation: Deficiency in Vitamin B12 leads to elevated homocysteine levels, which is associated with increased oxidative stress. This oxidative stress, in turn, is known to make omega-3 fatty acids like DHA more susceptible to degradation. Presence of Vitamin B-12 improves the oxidative stress of DHA.
Solubility: This study does not have much information on impacts on solubility.
Stability: As per the study, it is found that vitamin B12 is significant in protecting DHA's integrity.
Micronutrients: The specific effects of Vitamin B12 on the Micronutrients of DHA is not discussed in this study.
Summary: Vitamin B12 plays a pivotal role in protecting DHA from oxidative degradation by managing homocysteine levels and reducing oxidative stress, underscoring the importance of adequate vitamin B12 levels for brain health and the integrity of omega-3 fatty acids.

For example -  if the website is https://www.sciencedirect.com/science/article/abs/pii/S0952327815000885, it mentions the below impacts:

Oxidation: Direct effects on DHA's oxidation nature are not outlined due to Vitamin B12.
Solubility: This study does not have much information on impacts on solubility.
Stability: Vitamin B12 influences lipid metabolic pathways. Vitamin B12 is a cofactor for the mitochondrial enzyme methyl malonyl CoA mutase, which regulates the rate of long-chain fatty acyl-CoA transfer into the mitochondria. This activity can lead to the accumulation of fatty acids in the cytosol, thereby influencing lipid metabolism.
Micronutrients: The specific effects of Vitamin B12 on the Micronutrients of DHA is not discussed in this study.
Summary: Although direct effects on DHA's oxidation nature are not outlined, the study's focus on examining the impacts of prenatal and postnatal diets, both deficient and supplemented with Vitamin B12 and omega-3 fatty acids, on lipid metabolism could inferentially suggest a regulatory role of Vitamin B12 on how DHA is metabolized and utilized in the body.

Some website might not contain the study on interaction of DHA with a compound. In such cases, please mention that this study does not done on DHA.
I will give you the another website. Please provide me info on impacts in above format.

Please go the website -  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5158039/. Please mention the impacts according to the website.

'''

In [ ]:
'''
You are a very helpful assisant.
It is very important to know the positive and negative effects due to interaction between DHA/docosahexaenoic acid/omega 3-fatty acid and other chemical compounds like vitamin B12, Vitamin E, Vitamin D etc.
Interaction between DHA and other chemical compounds impacts the oxidation, solubility, stability and microneutrients of the DHA.
I will give you the website link of research papers and article. You please go through the website and give me information on the impacts oxidation, solubility, stability and microneutrients.


For example -  if the website is https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4727338/, it mentions the below impacts:

Oxidation: Deficiency in Vitamin B12 leads to elevated homocysteine levels, which is associated with increased oxidative stress. This oxidative stress, in turn, is known to make omega-3 fatty acids like DHA more susceptible to degradation.
Solubility: This study does not have much information on impacts on solubility.
Stability: As per the study, it is found that vitamin B12 significant in protecting DHA's integrity.
Microneutrients: The specific effects of Vitamin B12 on the Microneutrients of DHA is not discussed in this study.
Summary: Vitamin B12 plays a pivotal role in protecting DHA from oxidative degradation by managing homocysteine levels and reducing oxidative stress, underscoring the importance of adequate vitamin B12 levels for brain health and the integrity of omega-3 fatty acids.

For example -  if the website is https://www.sciencedirect.com/science/article/abs/pii/S0952327815000885, it mentions the below impacts:

Oxidation: Direct effects on DHA's oxidation nature are not outlined due to Vitamin B12.
Solubility: This study does not have much information on impacts on solubility.
Stability: Vitamin B12 influences lipid metabolic pathways. Vitamin B12 is a cofactor for the mitochondrial enzyme methyl malonyl CoA mutase, which regulates the rate of long-chain fatty acyl-CoA transfer into the mitochondria. This activity can lead to the accumulation of fatty acids in the cytosol, thereby influencing lipid metabolism.
Microneutrients: The specific effects of Vitamin B12 on the Microneutrients of DHA is not discussed in this study.
Summary: Although direct effects on DHA's oxidation nature are not outlined, the study's focus on examining the impacts of prenatal and postnatal diets, both deficient and supplemented with Vitamin B12 and omega-3 fatty acids, on lipid metabolism could inferentially suggest a regulatory role of Vitamin B12 on how DHA is metabolized and utilized in the body.

Some website might not contain the study on interaction of DHA with a compound. In such cases, please mention that this study does not done on DHA.
I will give you another website. Please provide me your response on impacts on DHA due to interaction with Vitamin D in mentioned above in tabular format.
please put the 5 information areas - Oxidation, Solubility, Stability, Micronutrients, and Summary in a tabular format for each website. While doing so please ensure that your analysis and the report continues to be thorough and detailed.

Please go the website - https://link.springer.com/article/10.1007/s00125-020-05300-7/. Please mention the impacts according to the website.

'''

In [7]:
sorted(['k', 'p','a'])

['a', 'k', 'p']

In [ ]:
'''You are a very helpful assistant.
It is very important to know the positive and negative effects due to interaction between Ferrous Sulphate/iron and polyphenols, phenolic acid, gallic acid or tannic acid.

Due to Interaction between FeSO4 or Ferrous Sulphate and polyphenols, polyphenols impacts the  Oxidations, Reductions, Precipitation, Absorption, Degradation, Agglomeration, Off-notes, Colour fading, Colour changing properties of Ferrous Sulphate.

I will give you the website link of research papers and articles.You please go through the website and give me information on the interactions between FeSO4 and Polyphenols with respect to the Oxidations, Reductions, Precipitation, Absorption, Degradation,
Agglomeration, Off-notes, Colour fading, Colour changing properties of Ferrous Sulphate and the corresponding Impact as well as the Probability of that Impact occurring.

For the Impact itself, please surmise in the form a scale of Minimal, Medium and High.
Similary For the Probability of this Impact to occur, please surmise your conclusion on a scale of Low, Medium and High.

Please look for the words like could, indicates, indicating, infer, may, might, points to, potential, potentially, suggest, suggests, semblance, chance, possible, possibility, likely, probable, indirectly, indirect are indicators that the study in website contains indication of probability. Based on the words, please make sure to give the probability in scale of Low, Medium and High.
Similarly you please give the impacts in the form a scale of Minimal, Medium and High.

Please give the detailed analysis of each impacts in tabular format alog with impact level and probability level.
Please also give a 4-5 line overall summary of the article in the website.

Some website might not contain the study on interaction of Ferrous Sulphate and polyphenols. In such cases, please mention that this study does not done on Ferrous Sulphate.

Please go the website - https://pubs.acs.org/doi/10.1021/acs.jafc.7b05726. Please give the report according to the website.
'''



In [ ]:
'''You are a very helpful assistant.
A website of research paper mainly contains the study of interactions between main compound (iron or Ferrous sulphate or ferrous) and other interaction compounds. I would like to know the main compound
and all other compounds which are used to done the study against the main compound in a research paper.

please know that interaction compounds can be only from the list of ['whole milk powder', 'cocoa powder', 'sugar', 'maltodextrin', 'flavors', 'tocopheryl acetate (vitamin e)',
'niacinamide (vitamin b3)', 'vitamin d3 powder ', 'vitamin a palmitate', 'phytonadione (vitamin k1)', 'thiamine hydrochloride (vitamin b1)', 'calcium pantothenate  (vitamin b5)',
'd-biotin  (vitamin b7)', 'riboflavin (vitamin b2)', 'vitamin b12', 'pyridoxine hydrochloride (vitamin b6)', 'folic acid  (vitamin b9)', 'zinc sulfate', 'zinc', 'manganese sulphate','manganese',
'cupric sulfate powder', 'copper', 'sodium molybdate dihyd','sodium', 'potassium iodide', 'potassium', 'sodium selenite anhydrous', 'chromic chloride hexahydrate', 'tricalcium phosphate']

I also would like to know the all the areas of study mentioned in paper.
Please know that areas of study can be only from the list of ['Interaction', 'Reaction', 'Oxidation', 'Reduction', 'Preciptation', 'Absorption','Degradation', 'Aglomeration', 'Off-notes', 'Colour fading', 'Colour changing']

For example - if the website is https://link.springer.com/article/10.1007/s12013-009-9043-x, it mentions the below infomation:

Main Compound: Iron or Ferrous sulphate
Interaction compounds: Polyphenol
Area of study: Oxidation, Absorption, degradation, Reduction, 'Colour changing', 'Colour fading', 'Interaction'

For example - if the website is https://www.sciencedirect.com/science/article/pii/S0141391017302008, it mentions the below infomation:

Main Compound: Iron or Ferrous sulphate
Interaction compounds: Gallic acid
Area of study: Oxidation, degradation, 'Colour changing', 'Colour fading'

I will give you another website. Please give me resposne in same format as mentioned above.
Please remember that area of study can be only the above mentioned list of areas of study. If a website does not mention the area of study as per in the list, please give area of study as not available.

Also remember that interaction compounds can be only the above mentioned list of interaction compounds. Website might contain compounds very similar to the list. Please give if similar compounds are mentioned in website. If interaction compounds are not in list or similar to the ones present in list, please give not available.

Please go the website - https://www.sciencedirect.com/science/article/pii/S0002916523433680. Please give the report according to the website.
'''